In [1]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__: owner: {owner_class}, property_name: {property_name}")

In [2]:
class Person:
    name = ValidString() # calls the set name method when compiling

__set_name__: owner: <class '__main__.Person'>, property_name: name


we can save property name in descriptor instance dictionary

In [9]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__: owner: {owner_class}, property_name: {property_name}")
        self.property_name = property_name
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print(f"__get__ called for property {self.property_name} of instance {instance}")

In [10]:
class Person:
    name = ValidString()

__set_name__: owner: <class '__main__.Person'>, property_name: name


In [11]:
Person.name.__dict__

{'property_name': 'name'}

In [13]:
Person.name.property_name

'name'

In [14]:
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__: owner: <class '__main__.Person'>, property_name: first_name
__set_name__: owner: <class '__main__.Person'>, property_name: last_name


In [15]:
p = Person()
p.first_name

__get__ called for property first_name of instance <__main__.Person object at 0x0000021333010B88>


#### let's finish implementing the class

this datadescriptor uses the instance dictionary to store the property. we are assuming that instances are not gonna be limited by slots (or one of the slots is `__dict__`)

In [20]:
class ValidString:
    def __init__(self, min_length=None):
        self.min_length = min_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f"{self.property_name} must be a String")
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"{self.property_name} must be at least {self.min_length} characters.")
        key = "_" + self.property_name
        setattr(instance, key, value)
        
    
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__: owner: {owner_class}, property_name: {property_name}")
        self.property_name = property_name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        key = "_" + self.property_name
        return getattr(instance, key, None)


In [21]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

__set_name__: owner: <class '__main__.Person'>, property_name: first_name
__set_name__: owner: <class '__main__.Person'>, property_name: last_name


In [22]:
p = Person()

In [24]:
try:
    p.first_name = "Alex"
    p.last_name = "g"
except ValueError as ex:
    print(ex)

last_name must be at least 2 characters.


In [25]:
p.first_name, p.last_name

('Alex', None)

In [26]:
p.first_name = "Alex"
p.last_name = "giant"
p.first_name, p.last_name

('Alex', 'giant')

In [27]:
p.__dict__

{'_first_name': 'Alex', '_last_name': 'giant'}

#### there is still an issue that we could overwrite existing attribute doing
` key = "_" + self.property_name`

In [28]:
p._first_name = "some data i want to store"

In [29]:
p.__dict__

{'_first_name': 'some date i want to store', '_last_name': 'giant'}

In [30]:
p.first_name, p.last_name

('some date i want to store', 'giant')

In [31]:
p.first_name = "Alex"
p.__dict__, p.first_name, p.last_name

({'_first_name': 'Alex', '_last_name': 'giant'}, 'Alex', 'giant')

#### how to store the value in the instance using the exact same name?

how to avoid situation when instance attribute shadows class attribute

In [32]:
class BankAccount:
    apr = 10
b = BankAccount()
b.apr, b.__dict__

(10, {})

In [34]:
b.apr = 100
b.apr, b.__dict__, BankAccount.apr

(100, {'apr': 100}, 10)

#### if we try and store inside first_name at the instance level, is that going to shadow the first_name which is now a descriptor object at the class level?

#### it will work OK with data descriptor

In [41]:
class ValidString:
    def __init__(self, min_length=None):
        self.min_length = min_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f"{self.property_name} must be a String")
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"{self.property_name} must be at least {self.min_length} characters.")
# we cannot use setattr here, cause it will call this __set__ method 
# (cause setattr will be the same as instance.first_name = value)
# and we will have infinite recursion
        instance.__dict__[self.property_name] = value
        
    
    def __set_name__(self, owner_class, property_name):
        print(f"__set_name__: owner: {owner_class}, property_name: {property_name}")
        self.property_name = property_name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print("__get__ called")
        return instance.__dict__.get(self.property_name, None)


In [42]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

__set_name__: owner: <class '__main__.Person'>, property_name: first_name
__set_name__: owner: <class '__main__.Person'>, property_name: last_name


In [43]:
p = Person()
p.__dict__

{}

In [44]:
p.first_name = "Alex"

In [45]:
p.__dict__

{'first_name': 'Alex'}

In [46]:
p.first_name

__get__ called


'Alex'

In [48]:
p2 = Person()
p2.first_name = "Hop"
p2.__dict__

{'first_name': 'Hop'}

In [50]:
p2.first_name, p.first_name

__get__ called
__get__ called


('Hop', 'Alex')

In [51]:
p2.__dict__["first_name"]="store"

In [52]:
p2.first_name

__get__ called


'store'

In [53]:
p2.first_name = "333"

In [54]:
p2.first_name

__get__ called


'333'